In [ ]:
#!pip install tensorflow-gpu
!pip3 install matplotlib
!pip3 install keras
#!dir |more
#!pip install google.coLab
#from google.coLab import drive
#import google as ggl
#print(ggl.__doc__)
#drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/Class 1.zip"

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook

import numpy as np
import glob
import pathlib
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes


from PIL import Image
import timeit
imgspec = {}
imgspec["size"]= 64
imgspec["channels"]=1
imgspec["len"] = (imgspec["size"]**2)*imgspec["channels"]
imgspec["descArr"]=[imgspec["size"],imgspec["size"],imgspec["channels"]]
print(imgspec)
print(imgspec["descArr"])
imgspec["descArr"][0:2]

device = 'DML'
#device='/device:XLA_CPU:0'
#device = '/cpu'
floattype = tf.float32
tf.keras.backend.set_floatx('float32')

#local_device_protos = tf.python.client.device_lib.list_local_devices()
#print([x.name for x in local_device_protos])

####################
num=5990/1
sets = [1,1,1,1,1,1]
###################
#sets = ['1','4']
def boolActivation(x):
    return x>.5

def SaveImgOut(tensor,idx,execute=True):
    #print (type(y_test))
    with tf.device(device):
      #print(tensor.shape)
      #img = tf.reshape(X_test[0],[imgspec["len"]])
      
      #img = np.array([img.numpy()])
      #img = img.numpy()
      #print(tensor)
      if execute:        
        #tensor = tf.reshape(tensor,imgspec["descArr"])
        tensor = tf.reshape(tensor,[1,imgspec["len"]])
        tensor=model(tensor)
        
      #print (tensor)
      img=tf.reshape(tensor,imgspec["descArr"]).numpy()
      tf.keras.preprocessing.image.save_img('./'+str(idx)+'.png',img)
        
def getDataSet(num,cls=['1']):

    def decode_img(img):        
        #print("decode")
        with tf.device(device):
            img= tf.keras.preprocessing.image.load_img(img,color_mode="grayscale")
            img = img.resize((imgspec["size"], imgspec["size"]), Image.LANCZOS )
            img = tf.keras.preprocessing.image.img_to_array(img)
            
            #img =tf.image.decode_png(img, channels=imgspec["channels"],dtype=tf.uint8)   
            #print(img.shape)
            #img = tf.reshape(img,[imgspec["len"] ])
            #print(img.shape)
            #print(type(img))
            #img=tf.keras.preprocessing.image.resize
            img = tf.image.convert_image_dtype(img, floattype)
            
            #img = tf.constant( img) 
            #print (img)
            
            img=tf.reshape (img,[4096])
            #print(img.shape)
            #print(img[0].shape)
            return img

    def LoadOne(pathpair):
        # print(pathpair)
        # label = get_label(file_path)
        # load the raw data from the file as a string
        label = pathpair[1][0]
        img = pathpair[0][0]

        with tf.device(device):
            label = decode_img(label)
            img   = decode_img(img)
            #print (img)
            return [img, label]


    
    dataset=[]
    print(cls)
    numClasses=len(cls)
    for iclass in range(numClasses):
        print('Processing class '+str(iclass)+', weight: '+str(cls[iclass]))
        data_dir = pathlib.Path('./Mask/Class '+str(iclass+1))
        print('searching: '+str(data_dir))
        files = list(data_dir.glob('*.png'))
        print(len(files))

        data = [ [[str(i)],[str(i).replace("x.png","y.png")]]  for i in files if str(i).find("y.png")==-1 ]
        print('found files: ',len(data))
        
        npcls=np.array(cls)
        #print(npcls)
        classWeight= cls[iclass]/npcls.sum()
        print('perWeight: '+str(classWeight))
        needed = int(classWeight*(num))
        print('Needed: ',needed)
        for i in range(needed):
            #print(i)
            n=LoadOne(data[i])
            dataset.append(n)
            #Xtrain.append(tf.reshape(n[0],[imgspec["len"]]))
            #ytrain.append(tf.reshape(n[1],[imgspec["len"]]))
            
        #labeled_ds = tf.data.Dataset.from_generator(Gen,(tf.float32,tf.float32))
    #dataset= tf.data.Dataset.from_tensor_slices(dataset)
    return tf.data.Dataset.from_tensor_slices(dataset),len(dataset)

X_train=[]
y_train=[]

tsize = int(min(100,int(num*(3/4.0))))

allData,ln= getDataSet(num,sets)
allData= allData.shuffle(ln)

#print(len(allData))
#print(num,tsize,len(allData))
print(np.array(allData).shape)
for record in allData:
    #print(record[0])
    X_train.append(record[0])
    y_train.append(record[1])
    
print(len(X_train))
print(len(y_train))

#X_train = allData[0][0:num-tsize]
#y_train = allData[1][0:num-tsize]

#X_test = allData[0][num-tsize:num]
#y_test = allData[1][num-tsize:num]

SaveImgOut(X_train[10],0,execute=False)
SaveImgOut(y_train[10],1,execute=False)

#print('x')
#print(X_test)   
#print('y')
#print(y_test)   
#for image, label in thing:
    #print ("loop")
    #print("Image shape: ", image.numpy().shape)
    #print("Label: ", label.numpy().shape)
    #plt.imshow(image)

In [ ]:
def MakeModel(u,t):
    is2d=False
    with tf.device(device):
        model = tf.keras.models.Sequential()
        #model.add(tf.keras.layers.Dense(units=256 , activation='relu' ,input_shape=(256*256*3,)))
        #model.add(tf.keras.layers.Dense(units=16 , activation='relu'))
        #model.add(tf.keras.layers.Dense(units=32 , activation='relu',input_shape=(16,)))
        #model.add(tf.keras.layers.Dense(units=64 , activation='relu',input_shape=(32,)))
        #model.add(tf.keras.layers.Dense(units=196608 , activation='hard_sigmoid',input_shape=(64,)))    
        #model.add(tf.keras.layers.Dense(units=imgspec["len"] , activation='relu'))


        for i in range(u[0]):
            
            if t[i]=='de' and is2d:                
                model.add(tf.keras.layers.Flatten())
                
            if t[i]=='de':
                model.add(tf.keras.layers.Dense(units=u[1] , activation=tf.keras.activations.selu))
                print('Adding DENSE layer: Units='+str(u[1]))
            
            if t[i][:2]=='co' and not is2d:
                model.add(tf.keras.layers.Reshape((64,64,1)))
                is2d=True
                
            if t[i][:2]=='co' and is2d:
                kernel = int(t[i][2])
                kernel= (kernel,kernel)
                
                model.add(tf.keras.layers.Conv2D(1,kernel, activation=tf.keras.activations.selu))
                print('Adding CONV2D layer: Units='+str(u[1]))
            
            if t[i]=='po':
                pass



        model.add(tf.keras.layers.Dense(units=imgspec["len"] , activation= 'sigmoid'))
        return model
        

In [ ]:
models=[]
models.append(MakeModel( [8,128] ,['de']*8) ) 
models.append(MakeModel( [8,128] ,['co3']*1+['de']*7) ) 
models.append(MakeModel( [8,128] ,['co3']*4+['de']*4) ) 
#models.append(MakeModel( [1,128] ,['de']*1) ) 


for model in models:
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=.00025),
              loss=tf.losses.absolute_difference )
    
print(len(models))

In [ ]:
from IPython.display import clear_output
#print(model.optimizer.variables())

with tf.device(device):
    X_train = tf.convert_to_tensor(X_train)
    y_train = tf.convert_to_tensor(y_train)
    X_train=tf.cast(X_train, floattype)
    y_train=tf.cast(y_train, floattype)

with tf.device(device):
   
    #SaveImgOut(X_train[0],i,execute = False)
    #SaveImgOut(X_train[0],i+100000,execute = True)
    
    #SaveImgOut(y_test[0],123123,execute=False)
    
    lossHist=[[]]*len(models)
    valHist =[[]]*len(models)
    
    print(lossHist)
    print(valHist)
    while True:
        for modelIndex, model in enumerate(models):
            #model.summary()
            #en=np.min([i+1,5])    
            en=1
            print('modelidx',modelIndex)
            
            import time

            t_end = time.time() + 5
            while time.time() < t_end:
                history=[]
                #print('x.shape:',X_train.shape)
                #print('y.shape:',y_train.shape)
                #print(X_train[0].shape)
                #print(y_train[0].shape)
                #print(type(X_train[0]))
                history = model.fit(X_train,y_train,validation_split=.5,verbose=1,
                                    batch_size=40,shuffle=False)
                curLoss = history.history["loss"]
                curVal  = history.history["val_loss"]
                lossHist[modelIndex]=lossHist[modelIndex]+[curLoss]
                valHist [modelIndex]=valHist[modelIndex]+[curVal]

            print ('Model: '+str(modelIndex))
            #print(lossHist[modelIndex])
            
            #historys[modelIndex].append(overallHist)

            

            #print(history.history)
            print(str(0)+"curLoss: "+ str(curLoss))
            print(str(0)+"curVal: " + str(curVal))

            imgidx = 10
            testT = X_train[imgidx]
            print(type(testT))
            print(testT)
            
            #result = model.call(testT)
            result = model(tf.reshape(testT,[1,imgspec["len"]]))
            result = model.predict(result)
            

            img1 = tf.reshape(result,imgspec["descArr"] )
            img2 = tf.reshape(X_train[imgidx],imgspec["descArr"])
            img3 = tf.reshape(y_train[imgidx],imgspec["descArr"])

            img1=tf.concat(values=[img1,img2,img3],axis=1)


            fig,ax = plt.subplots(1,1)
            #print(dir(fig))
            fig.set_figwidth(10)
            ax.imshow(img1,interpolation='nearest')
            #plt.show()

           
        
            #print (overallHist[0])
            #print (overallHist[1])
            #print(historys)
            #print(historys[0])
        clear_output(wait=True)    
        fig2,ax = plt.subplots(1,1)        
        fig2.set_figwidth(10)

        for i in range(len(lossHist)):            
            print ('Model: '+str(i))
            
            #l = historys[i][0]
            #v = historys[i][1]
            ax.plot(lossHist[i][5:],label=str(i)+" Loss",linestyle='dashed')
            ax.plot(valHist [i][5:],label=str(i)+" Validation")
            ax.legend()
            #ax.plot(v)

        plt.show()  

        
        

In [ ]:
#print (y_test.numpy())
#print (y_train)
with tf.device("/gpu:0"):
    X_test = tf.convert_to_tensor(X_test)
    y_test = tf.convert_to_tensor(y_test)
    test_loss = model.evaluate(X_test.numpy(), y_test.numpy())

In [ ]:
GetImgOut(X_test[0])

In [ ]:
a=np.array([X_test[3]])
with tf.device("/gpu:0"):
    b=model.call(a)
with tf.device("/gpu:0"):
    plt.imshow(tf.reshape(a,))
    plt.show()
with tf.device("/gpu:0"):
    plt.imshow(tf.reshape(b,))
    plt.show()

In [ ]:
model.save("model")

In [ ]:
#
import tensorflow as tf
model =tf.keras.models.load_model("model")

In [ ]:
model.summary()